# OUR TREM NAME

## Project: The Advertising Firm
![](https://upload.wikimedia.org/wikipedia/en/7/7b/Don_Draper_Wiki.jpg)

We are an ad agency working on personalizing ads, so that you aren't bothered by ads you don’t care about, and only see stuff you may be interested in. 

We are promoting a product that people with income <50k are not interested in. 
**We want to market to people with income greater then $50,000**

We start with census data (94-95) on demographics, age, education, country, occupation, income, etc.
The data was split into train/test in approximately 2/3, 1/3 split.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import data_prep
ROOT_DIR = os.path.dirname('data/')

train_file_path = os.path.abspath(os.path.join(ROOT_DIR, 'census-income.csv'))
data1=data_prep.load_file(train_file_path)
train_df=data_prep.clean_data(data1)

test_file_path = os.path.abspath(os.path.join(ROOT_DIR, 'census-income-test.csv'))
data2=data_prep.load_file(test_file_path)
test_df=data_prep.clean_data(data2)

In [3]:
train_df.head(3)

,target,detailed_hsld_smry_1,region_prev_res_1,marital_status_1,marital_status_2,worker_class_1,worker_class_2,worker_class_3,age,wage,...,education_2,migration_1,detailed_occ_cd_A,detailed_occ_cd_B,detailed_occ_cd_C,detailed_occ_cd_D,detailed_ind_cd_A,detailed_ind_cd_B,detailed_ind_cd_C,detailed_ind_cd_D
0,0,0,0,0,0,0,0,0,58,0,...,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0,0,0,0,0,0,0,0,18,0,...,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0,0,0,0,0,0,0,0,9,0,...,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [4]:
test_df.head(3)

,target,detailed_hsld_smry_1,region_prev_res_1,marital_status_1,marital_status_2,worker_class_1,worker_class_2,worker_class_3,age,wage,...,education_2,migration_1,detailed_occ_cd_A,detailed_occ_cd_B,detailed_occ_cd_C,detailed_occ_cd_D,detailed_ind_cd_A,detailed_ind_cd_B,detailed_ind_cd_C,detailed_ind_cd_D
0,0,0,0,0,0,0,0,0,44,0,...,0,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0,0,0,0,0,0,0,0,2,0,...,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0,0,0,0,0,0,0,0,35,0,...,0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
